<div dir="rtl" align="right">

# ایجنت جستجوی وب و اطلاعات با سه ابزار
در این بخش، یک ایجنت هوشمند ساخته‌ایم که می‌تواند به سوالات فارسی شما با استفاده از سه ابزار زیر پاسخ دهد:

- **ماشین حساب:** برای انجام محاسبات ریاضی ساده و سریع
- **ویکی‌پدیا:** برای دریافت اطلاعات و خلاصه‌های فارسی از دانشنامه ویکی‌پدیا
- **جستجوی گوگل با ترجمه خودکار:** برای دریافت جدیدترین اخبار و اطلاعات از وب، حتی اگر جستجوی فارسی در گوگل محدود باشد

ابزار جستجوی گوگل به صورت زیر عمل می‌کند:
1. سؤال فارسی شما ابتدا به انگلیسی ترجمه می‌شود تا نتایج جستجو دقیق‌تر و مرتبط‌تر باشند.
2. جستجو با SerpAPI و گوگل انجام می‌شود.
3. خلاصه نتایج به فارسی ترجمه و نمایش داده می‌شود.

این ساختار باعث می‌شود ایجنت بتواند بهترین ابزار را برای هر سؤال انتخاب کند و پاسخ‌هایی کاربردی و قابل فهم ارائه دهد.

</div>

<div dir="rtl" align="right">

## سناریو: دستیار هوشمند با سه ابزار
در این سناریو، ایجنت می‌تواند به سوالات فارسی شما با توجه به نوع سؤال، بهترین ابزار را انتخاب کند:

- اگر سؤال شما محاسبات ریاضی باشد، ماشین حساب پاسخ می‌دهد.
- اگر سؤال شما درباره اطلاعات عمومی یا دانش باشد، ویکی‌پدیا جستجو می‌شود.
- اگر سؤال شما نیاز به جستجوی اخبار یا اطلاعات به‌روز دارد، ابزار جستجوی گوگل با ترجمه خودکار استفاده می‌شود.

مثال:
- "آخرین اخبار هوش مصنوعی چیست؟" (جستجوی وب با ترجمه)
- "جمعیت ایران در سال ۲۰۲۴ چقدر است؟" (جستجوی وب یا ویکی‌پدیا)
- "حاصل ۱۲۳۴ تقسیم بر ۲۳ چند می‌شود؟" (ماشین حساب)
- "چه کسی رئیس جمهور فعلی فرانسه است؟" (جستجوی وب با ترجمه)

</div>

<div dir="rtl" align="right">

## ابزار جستجوی گوگل با ترجمه خودکار
این ابزار به گونه‌ای طراحی شده که اگر سؤال شما نیاز به جستجوی وب داشته باشد، ابتدا سؤال فارسی را به انگلیسی ترجمه می‌کند، سپس با SerpAPI و گوگل جستجو انجام می‌دهد و خلاصه نتایج را مجدداً به فارسی ترجمه و نمایش می‌دهد.

این روش باعث می‌شود محدودیت‌های جستجوی فارسی در گوگل رفع شود و پاسخ‌ها مرتبط‌تر و قابل فهم‌تر باشند. ایجنت به طور خودکار تشخیص می‌دهد که برای هر سؤال کدام ابزار مناسب‌تر است و بهترین پاسخ را ارائه می‌دهد.

</div>

In [12]:
from dotenv import load_dotenv
import os

from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool
from langchain.agents import initialize_agent
from langchain_community.utilities import WikipediaAPIWrapper
import serpapi

# loading environment variables
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")

if not os.getenv("SERPAPI_API_KEY"):
    raise ValueError("SERPAPI_API_KEY در فایل .env تعریف نشده است!")

# LLM 
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

# Calculator tool 
def calculator(expression: str) -> str:
    try:
        return str(eval(expression))
    except Exception as e:
        return f"خطا: {e}"

# Wikipedia search tool
wiki = WikipediaAPIWrapper()
wikipedia = Tool.from_function(
    func=wiki.run,
    name="wikipedia_search",
    description="جستجو در ویکی‌پدیا برای اطلاعات مرتبط."
)

# serpapi.Client for Google search
api_key = os.getenv("SERPAPI_API_KEY")
serp_client = serpapi.Client(api_key=api_key)

def google_search(query: str) -> str:
    result = serp_client.search(
        engine="google",
        q=query,
        location="United States",
        hl="fa",
        gl="ir",
        num=5
    )
    organic = result.get("organic_results", [])
    return "\n".join(f"- {r.get('title')}: {r.get('link')}" for r in organic)

google_tool = Tool.from_function(
    func=google_search,
    name="google_search",
    description="جستجو در وب با استفاده از SerpApi (GoogleSearch)."
)

tools = [
    Tool.from_function(func=calculator, name="calculator", description="محاسبه عبارات ریاضی."),
    wikipedia,
    google_tool,
]

agent = initialize_agent(tools=tools, llm=llm, agent="openai-functions", verbose=True)

examples = [
    "آخرین اخبار هوش مصنوعی چیست؟",
    "جمعیت ایران در سال ۲۰۲۴ چقدر است؟",
    "حاصل ۱۲۳۴ تقسیم بر ۲۳ چند می‌شود؟"
]

for q in examples:
    print(f"\n❓ سؤال: {q}")
    resp = agent.run(q)
    print("✅ پاسخ:", resp)


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



❓ سؤال: آخرین اخبار هوش مصنوعی چیست؟

Invoking: `google_search` with `آخرین اخبار هوش مصنوعی`



Invoking: `google_search` with `آخرین اخبار هوش مصنوعی`


- هوش مصنوعی: https://digiato.com/topic/tech/artificial-intelligence
- اخبار هوش مصنوعی - تازه های ایران و جهان: https://hooshio.com/news/
- هوش مصنوعی - خبرگزاری مهر | اخبار ایران و جهان: https://www.mehrnews.com/tag/%D9%87%D9%88%D8%B4+%D9%85%D8%B5%D9%86%D9%88%D8%B9%DB%8C- هوش مصنوعی: https://digiato.com/topic/tech/artificial-intelligence
- اخبار هوش مصنوعی - تازه های ایران و جهان: https://hooshio.com/news/
- هوش مصنوعی - خبرگزاری مهر | اخبار ایران و جهان: https://www.mehrnews.com/tag/%D9%87%D9%88%D8%B4+%D9%85%D8%B5%D9%86%D9%88%D8%B9%DB%8C

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


با توجه به جستجوی انجام شده، می‌توانید آخرین اخبار هوش مصنوعی را در لینک‌های زیر مشاهده کنید:
1. [هوش مصنوعی - دیجیاتو](https://digiato.com/topic/tech/artificial-intelligence)
2. [اخبار هوش مصنوعی - هوشیو](https://hooshio.com/news/)
3. [هوش مصنوعی - خبرگزاری مهر](https://www.mehrnews.com/tag/%D9%87%D9%88%D8%B4+%D9%85%D8%B5%D9%86%D9%88%D8%B9%DB%8C)

> Finished chain.
✅ پاسخ: با توجه به جستجوی انجام شده، می‌توانید آخرین اخبار هوش مصنوعی را در لینک‌های زیر مشاهده کنید:
1. [هوش مصنوعی - دیجیاتو](https://digiato.com/topic/tech/artificial-intelligence)
2. [اخبار هوش مصنوعی - هوشیو](https://hooshio.com/news/)
3. [هوش مصنوعی - خبرگزاری مهر](https://www.mehrnews.com/tag/%D9%87%D9%88%D8%B4+%D9%85%D8%B5%D9%86%D9%88%D8%B9%DB%8C)

❓ سؤال: جمعیت ایران در سال ۲۰۲۴ چقدر است؟

Invoking: `wikipedia_search` with `جمعیت ایران در سال ۲۰۲۴`



Invoking: `wikipedia_search` with `جمعیت ایران در سال ۲۰۲۴`


Page: Fordo
Summary: Fordo (Persian: فُردو, Persian pronunciation: [foɾˈduː] or [foɾˈdo]) is a village i

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


متاسفانه اطلاعات دقیقی در مورد جمعیت ایران در سال ۲۰۲۴ در نتایج جستجوی ویکی‌پدیا موجود نیست. اگر مایل هستید، می‌توانم برای شما در وب جستجو کنم یا از منبع دیگری اطلاعات را به دست آورم.

> Finished chain.
✅ پاسخ: متاسفانه اطلاعات دقیقی در مورد جمعیت ایران در سال ۲۰۲۴ در نتایج جستجوی ویکی‌پدیا موجود نیست. اگر مایل هستید، می‌توانم برای شما در وب جستجو کنم یا از منبع دیگری اطلاعات را به دست آورم.

❓ سؤال: حاصل ۱۲۳۴ تقسیم بر ۲۳ چند می‌شود؟

Invoking: `calculator` with `1234 / 23`


53.65217391304348
Invoking: `calculator` with `1234 / 23`


53.65217391304348حاصل تقسیم ۱۲۳۴ بر ۲۳ برابر با ۵۳.۶۵ می‌شود.

> Finished chain.
✅ پاسخ: حاصل تقسیم ۱۲۳۴ بر ۲۳ برابر با ۵۳.۶۵ می‌شود.
حاصل تقسیم ۱۲۳۴ بر ۲۳ برابر با ۵۳.۶۵ می‌شود.

> Finished chain.
✅ پاسخ: حاصل تقسیم ۱۲۳۴ بر ۲۳ برابر با ۵۳.۶۵ می‌شود.


<div dir="rtl" align="right">

### نحوه عملکرد ایجنت و ابزارها
در سلول بالا، ایجنت شما سه ابزار دارد:
- ماشین حساب برای محاسبات ریاضی
- جستجوی ویکی‌پدیا برای دریافت اطلاعات فارسی
- جستجوی گوگل با ترجمه خودکار برای دریافت اخبار و اطلاعات به‌روز

کافیست سؤال خود را به فارسی وارد کنید. ایجنت به طور هوشمند تشخیص می‌دهد که کدام ابزار مناسب‌تر است و پاسخ را به صورت فارسی و قابل فهم نمایش می‌دهد. ابزار جستجوی گوگل با ترجمه خودکار، محدودیت‌های جستجوی فارسی را رفع می‌کند و نتایج دقیق‌تری ارائه می‌دهد.

</div>

In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool
from langchain.agents import initialize_agent
from dotenv import load_dotenv
import os
import serpapi

# Load API keys
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")

# LLM
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

# SerpAPI client
serp_client = serpapi.Client(api_key=os.getenv("SERPAPI_API_KEY"))

# Composite search tool: Persian input → English search → Persian response
def persian_google_search(query_fa: str) -> str:
    # Step 1: Translate Persian to English
    translation_prompt = f"Translate this Persian question to English (just the translation):\n{query_fa}"
    translated_query = llm.invoke(translation_prompt).content.strip()

    # Step 2: Search with translated query
    result = serp_client.search(
        engine="google",
        q=translated_query,
        location="United States",
        hl="en",
        gl="us",
        num=5
    )

    organic_results = result.get("organic_results", [])
    english_summary = "\n".join(f"- {r.get('title')}: {r.get('snippet')}" for r in organic_results if r.get("snippet"))

    if not english_summary:
        return "نتیجه‌ای یافت نشد."

    # Step 3: Translate result back to Persian
    summary_prompt = f"Please translate this English text to Persian:\n{english_summary}"
    translated_answer = llm.invoke(summary_prompt).content.strip()

    return translated_answer

# LangChain Tool wrapper
google_search_tool_fa = Tool.from_function(
    func=persian_google_search,
    name="persian_google_search",
    description="جستجو در وب با استفاده از گوگل. ورودی را به فارسی بدهید و خروجی را نیز به فارسی دریافت خواهید کرد."
)

# Initialize agent with just this tool (you can add more tools too)
agent = initialize_agent(
    tools=[google_search_tool_fa],
    llm=llm,
    agent="openai-functions",
    verbose=True
)

# Sample questions
questions = [
    "جمعیت ایران در سال ۲۰۲۴ چقدر است؟",
    "آخرین اخبار هوش مصنوعی چیست؟",
    "چه کسی رئیس جمهور فعلی فرانسه است؟"
]

for q in questions:
    print(f"\n❓ سؤال: {q}")
    answer = agent.run(q)
    print("✅ پاسخ:", answer)


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



❓ سؤال: جمعیت ایران در سال ۲۰۲۴ چقدر است؟

Invoking: `persian_google_search` with `جمعیت ایران در سال ۲۰۲۴`



Invoking: `persian_google_search` with `جمعیت ایران در سال ۲۰۲۴`


جمعیت ایران (2025): جمعیت ایران (2025 و تاریخی)؛ 2024، 91،567،738، 1.06٪؛ 2023، 90،608،707، 1.21٪؛ 2022، 89،524،246، 1.02٪؛ 2020، 87،723،443، 1.21٪ ...

جمعیت‌شناسی ایران: تا نوامبر 2024، جمعیت ایران حدود 91.5 میلیون نفر است. با این حال، در سال‌های اخیر، نرخ تولد ایران به طور قابل توجهی کاهش یافته است.

جمعیت ایران (2025) زنده: جمعیت‌شناسی ایران 2024. تا 1 ژانویه 2025، تخمین زده شده است که جمعیت ایران 88،874،624 نفر است. این افزایش 1.27٪ است (1،111،086 ...

جمعیت ایران 2025: این کشور دارای جمعیت 92.4 میلیون نفر است که آن را به عنوان 17مین کشور بزرگ جهان می‌سازد. پایتخت آن تهران است. اقتصاد ایران بر پایه نفت است و دارای تنوع ...جمعیت ایران (2025): جمعیت ایران (2025 و تاریخی)؛ 2024، 91،567،738، 1.06٪؛ 2023، 90،608،707، 1.21٪؛ 2022، 89،524،246، 1.02٪؛ 2020، 87،723،443، 1.21٪ ...

جمعیت‌شناسی ایران: تا نوامبر 2024

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


جمعیت ایران در سال ۲۰۲۴ حدود ۹۱.۵ میلیون نفر است.

> Finished chain.
✅ پاسخ: جمعیت ایران در سال ۲۰۲۴ حدود ۹۱.۵ میلیون نفر است.

❓ سؤال: آخرین اخبار هوش مصنوعی چیست؟

Invoking: `persian_google_search` with `آخرین اخبار هوش مصنوعی`



Invoking: `persian_google_search` with `آخرین اخبار هوش مصنوعی`


اخبار هوش مصنوعی | آخرین اخبار، تجزیه و تحلیل و رویدادهای هوش مصنوعی: هوش مصنوعی قوانین صنعت بیمه را بازنویسی می‌کند · IBM Power11 با معماری بدون توقف هدف از پذیرش هوش مصنوعی در شرکت‌هاست · هوش مصنوعی MedGemma باز Google ...
- هوش مصنوعی: آخرین اخبار و داستان‌های برتر درباره هوش مصنوعی، از جمله چت‌بات‌های هوش مصنوعی مانند ChatGPT شرکت مایکروسافت، چت‌بات هوش مصنوعی شرکت اپل و Bard گوگل.
- هوش مصنوعی - آخرین اخبار و تجزیه و تحلیل هوش مصنوعی: پوشش آخرین اخبار هوش مصنوعی با تمرکز بر فناوری، ابزارها و شرکت‌هایی که فناوری هوش مصنوعی را ایجاد می‌کنند.
- هوش مصنوعی | اخبار MIT | موسسه فناوری ماساچوست: سیستم هوش مصنوعی جدید زیرگروه‌های پنهان سلول را کشف می‌کند، پزشکی دقیق را تقویت می‌کند · تغییر در گف

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


بهترین راه برای دسترسی به آخرین اخبار هوش مصنوعی، مراجعه به سایت‌های معتبری مانند MIT و Google است. این سایت‌ها اخبار، تجزیه و تحلیل‌های جذاب و رویدادهای مرتبط با هوش مصنوعی را ارائه می‌دهند.

> Finished chain.
✅ پاسخ: بهترین راه برای دسترسی به آخرین اخبار هوش مصنوعی، مراجعه به سایت‌های معتبری مانند MIT و Google است. این سایت‌ها اخبار، تجزیه و تحلیل‌های جذاب و رویدادهای مرتبط با هوش مصنوعی را ارائه می‌دهند.

❓ سؤال: چه کسی رئیس جمهور فعلی فرانسه است؟
رئیس جمهور فعلی فرانسه امانوئل ماکرون است.

> Finished chain.
✅ پاسخ: رئیس جمهور فعلی فرانسه امانوئل ماکرون است.
رئیس جمهور فعلی فرانسه امانوئل ماکرون است.

> Finished chain.
✅ پاسخ: رئیس جمهور فعلی فرانسه امانوئل ماکرون است.


<div dir="rtl" align="right">

در این مثال، ایجنت می‌تواند سؤال فارسی شما را به انگلیسی ترجمه کند، با گوگل جستجو کند و پاسخ را مجدداً به فارسی نمایش دهد. کافیست سؤال خود را به فارسی وارد کنید تا پاسخ مرتبط و قابل فهم دریافت کنید.

</div>